In [1]:
#ライブラリをインポート
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from src import config
from pathlib import Path

In [2]:
# インポート元のフォルダを設定
folder_path = Path(config.DATA_PATH) # この部分を自分のフォルダに合わせて設定してください
try:
    csv_files = list(folder_path.glob('*/*.csv'))
    #print(f"read csv successfully: {csv_files}")
except:
    print("Import error!")

# CSVファイルをDataFrame形式に変換
list_of_dfs = []
labels = []

for file in csv_files:
    df = pd.read_csv(file)
    if "clean" in str(file): # clean: 正常
        label = pd.Series([0]*len(df))
    else:
        label = pd.Series([1]*len(df))
    list_of_dfs.append(df)
    labels.append(label)


X = pd.concat(list_of_dfs, ignore_index=True, axis=0)
y = pd.concat(labels, ignore_index=True, axis=0)

In [3]:
# データの確認(ここで、MLに入力できない特徴量を確認する)
# この場合だとtimeが扱えない
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 73 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   time                                     4200 non-null   object 
 1   cpu_usage_pct                            4200 non-null   float64
 2   ram_usage_pct                            4200 non-null   float64
 3   network_in                               4200 non-null   float64
 4   network_out                              4200 non-null   float64
 5   block:block_bio_backmerge                4200 non-null   int64  
 6   block:block_bio_remap                    4200 non-null   int64  
 7   block:block_dirty_buffer                 4200 non-null   int64  
 8   block:block_getrq                        4200 non-null   int64  
 9   block:block_touch_buffer                 4200 non-null   int64  
 10  block:block_unplug                       4200 no

In [4]:
# データの前処理
# 不要な特徴量(time)を削除
X = X.drop(columns=['time'])

# データを学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 正規化
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [5]:
# モデルのインポート(教師あり学習)
model = LogisticRegression
scores = []


clf = model()

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

# 結果の確認
score = clf.score(X_test, y_test)
scores.append(score)
print(f"Model name: {model.__name__}")
print(f"    accuracy: {score}\n")

Model name: LogisticRegression
    accuracy: 0.9166666666666666



/Users/daisukesugata/mldetect/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
